In [1]:
pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install datetime

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import xarray as xr
import h5py

from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.layers import Input, LeakyReLU, Dense, Activation, Flatten, Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, Reshape, Concatenate
from tensorflow.keras.models import Model

# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
with h5py.File('processed_data_ae/np_data.h5', 'r') as hf:
    data = hf['np_data'][:]

print(data.shape)

(8530, 192, 192, 2)


In [5]:
print(data)

[[[[-2.47433877e+00  4.50653076e-01]
   [-2.47227287e+00  4.22745228e-01]
   [-2.49654698e+00  3.96081567e-01]
   ...
   [-2.49194694e+00 -1.17015743e+00]
   [-2.54665089e+00 -1.16967881e+00]
   [-2.57032084e+00 -1.18266952e+00]]

  [[-2.52002358e+00  4.53049988e-01]
   [-2.53445411e+00  3.97361368e-01]
   [-2.54852366e+00  4.00280446e-01]
   ...
   [-2.58340573e+00 -1.12509418e+00]
   [-2.60144782e+00 -1.14579189e+00]
   [-2.60572982e+00 -1.15639758e+00]]

  [[-2.51759076e+00  4.58887845e-01]
   [-2.52549720e+00  4.21892971e-01]
   [-2.51069617e+00  4.61187005e-01]
   ...
   [-2.59329128e+00 -1.10369992e+00]
   [-2.60617042e+00 -1.12321818e+00]
   [-2.60261035e+00 -1.12687624e+00]]

  ...

  [[-1.58731222e+00 -9.34629977e-01]
   [-1.57048082e+00 -1.39101851e+00]
   [-1.55039418e+00 -1.14039171e+00]
   ...
   [-5.02201855e-01 -1.07426441e+00]
   [-5.04015684e-01 -1.18869841e+00]
   [-5.98666489e-01 -1.35226285e+00]]

  [[-1.65483463e+00 -6.58889890e-01]
   [-1.62351513e+00 -7.88042486e

In [6]:
print(np.max(data), np.min(data))

19.913855 -16.306587


In [7]:
#First split data into train+validation and test set
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

#Next split training again into train and validation
X_train, X_val = train_test_split(X_train, test_size=0.25, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(np.max(X_train), np.max(X_val), np.max(X_test), np.min(X_train), np.min(X_val), np.min(X_test))

(5118, 192, 192, 2)
(1706, 192, 192, 2)
(1706, 192, 192, 2)
19.913855 18.198473 19.736868 -16.306587 -16.134562 -16.26034


In [8]:
def encoder(latent_dim):
    '''
    return an encoder which encodes the input image into a latent vector with dimension latent_dim
    '''
    
    X_input = Input((192, 192, 2))
    
    #FIXME Should we add BN layer? I currently add that between conv and relu for the first 4 sets of layers
    X = Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding="same")(X_input)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    #FIXME Should we add some dropout layer to regularize the model? 
    #I didn't do that, but need to look at train/val error
    
    X = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = MaxPooling2D(pool_size=(2, 2), padding="same")(X)
    
    X = Flatten()(X)
    X = Dense(units=latent_dim)(X)
    #FIXME Should we add an activation layer here? I didn't do it
    
    model = Model(inputs = X_input, outputs = X)
    return model

In [9]:
def decoder(latent_dim):
    '''
    return an encoder which encodes the input image into a latent vector with dimension latent_dim
    '''
    
    X_input = Input((latent_dim))
    
    X = Dense(units=3*3*64, input_dim=latent_dim)(X_input)
    X = Reshape((3,3,64))(X)
    
    X = Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=32, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=32, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=16, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2DTranspose(filters=16, kernel_size=(3,3), strides=(2,2), padding="same")(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(filters=2, kernel_size=(3,3), strides=(1,1), padding="same")(X)    
    
    model = Model(inputs = X_input, outputs = X)
    return model

In [10]:
encoder_18_sub1 = encoder(18)
print(encoder_18_sub1.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 192, 192, 2)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 16)      304       
                                                                 
 batch_normalization (BatchN  (None, 192, 192, 16)     64        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 192, 192, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 96, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 16)        2320  

2022-07-12 12:24:35.442168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-12 12:24:35.781209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30965 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:01:00.0, compute capability: 7.0


In [11]:
decoder_18_sub1 = decoder(18)
print(decoder_18_sub1.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 18)]              0         
                                                                 
 dense_1 (Dense)             (None, 576)               10944     
                                                                 
 reshape (Reshape)           (None, 3, 3, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 6, 6, 64)         36928     
 nspose)                                                         
                                                                 
 batch_normalization_4 (Batc  (None, 6, 6, 64)         256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 6, 6, 64)          0   

In [12]:
class Autoencoder(Model):
    def __init__(self, encoder, decoder):
        super(Autoencoder, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [13]:
autoencoder_18 = Autoencoder(encoder_18_sub1, decoder_18_sub1)
autoencoder_18.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [14]:
logdir = "autoencoder_18/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_filepath = 'autoencoder_18/ckp/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               save_freq=10*40)

In [15]:
training_history = autoencoder_18.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=10000,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/10000


2022-07-12 12:26:24.968992: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


40/40 [==============================] - 9s 156ms/step - loss: 7.3824 - val_loss: 8.8248
Epoch 2/10000
40/40 [==============================] - 4s 105ms/step - loss: 3.8980 - val_loss: 5.7598
Epoch 3/10000
40/40 [==============================] - 4s 105ms/step - loss: 1.4100 - val_loss: 3.1422
Epoch 4/10000
40/40 [==============================] - 4s 106ms/step - loss: 1.1026 - val_loss: 2.4191
Epoch 5/10000
40/40 [==============================] - 4s 106ms/step - loss: 0.9721 - val_loss: 2.6529
Epoch 6/10000
40/40 [==============================] - 4s 105ms/step - loss: 0.8926 - val_loss: 2.0698
Epoch 7/10000
40/40 [==============================] - 4s 105ms/step - loss: 0.8718 - val_loss: 1.3960
Epoch 8/10000
40/40 [==============================] - 4s 106ms/step - loss: 0.7930 - val_loss: 1.3752
Epoch 9/10000
40/40 [==============================] - 4s 105ms/step - loss: 0.7894 - val_loss: 1.9782
Epoch 10/10000
40/40 [==============================] - 4s 107ms/step - loss: 0.7256 - 

In [24]:
class Autoencoder_hier_2(Model):
    def __init__(self, encoder, decoder, ae):
        super(Autoencoder_hier_2, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder
        self.ae = ae
        self.ae.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        latent_1 = self.ae.encoder(x)
        latent_all = Concatenate(axis=-1)([encoded, latent_1])
        decoded = self.decoder(latent_all)
        return decoded

In [25]:
encoder_18_sub2 = encoder(18)
print(encoder_18_sub2.summary())

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 192, 192, 2)]     0         
                                                                 
 conv2d_14 (Conv2D)          (None, 192, 192, 16)      304       
                                                                 
 batch_normalization_16 (Bat  (None, 192, 192, 16)     64        
 chNormalization)                                                
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 192, 192, 16)      0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 96, 96, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 96, 96, 16)        2320

In [26]:
decoder_18_sub2 = decoder(36)
print(decoder_18_sub2.summary())

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 36)]              0         
                                                                 
 dense_5 (Dense)             (None, 576)               21312     
                                                                 
 reshape_2 (Reshape)         (None, 3, 3, 64)          0         
                                                                 
 conv2d_transpose_12 (Conv2D  (None, 6, 6, 64)         36928     
 Transpose)                                                      
                                                                 
 batch_normalization_20 (Bat  (None, 6, 6, 64)         256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 6, 6, 64)          0   

In [27]:
autoencoder_18_hier_2 = Autoencoder_hier_2(encoder_18_sub2, decoder_18_sub2, autoencoder_18)
autoencoder_18_hier_2.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [28]:
logdir = "autoencoder_18_hier_2/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_filepath = 'autoencoder_18_hier_2/ckp/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               save_freq=10*40)

In [30]:
training_history_hier_2 = autoencoder_18_hier_2.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=20,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/20
40/40 [==============================] - 7s 137ms/step - loss: 7.4599 - val_loss: 6.0756
Epoch 2/20
40/40 [==============================] - 5s 116ms/step - loss: 3.4401 - val_loss: 1.9163
Epoch 3/20
40/40 [==============================] - 5s 116ms/step - loss: 1.1507 - val_loss: 1.3046
Epoch 4/20
40/40 [==============================] - 5s 119ms/step - loss: 0.8117 - val_loss: 1.0366
Epoch 5/20
40/40 [==============================] - 5s 116ms/step - loss: 0.6742 - val_loss: 0.9500
Epoch 6/20
40/40 [==============================] - 5s 117ms/step - loss: 0.6225 - val_loss: 1.0118
Epoch 7/20
40/40 [==============================] - 5s 116ms/step - loss: 0.6166 - val_loss: 0.7534
Epoch 8/20
40/40 [==============================] - 5s 115ms/step - loss: 0.5631 - val_loss: 0.6456
Epoch 9/20
40/40 [==============================] - 5s 117ms/step - loss: 0.5598 - val_loss: 0.4883
Epoch 10/20
40/40 [==============================] - 5s 117ms/step - loss: 0.5297 - val_loss: 0.4785

In [31]:
print(autoencoder_18.evaluate(X_test, X_test))

54/54 [==============================] - 1s 11ms/step - loss: 0.2165
0.21648062765598297


In [32]:
print(autoencoder_18_hier_2.ae.evaluate(X_test, X_test))

54/54 [==============================] - 1s 11ms/step - loss: 0.2165
0.21648062765598297


In [34]:
training_history_hier_2 = autoencoder_18_hier_2.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=5000,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.4685 - val_loss: 0.4719
Epoch 2/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.4297 - val_loss: 0.3923
Epoch 3/5000
40/40 [==============================] - 5s 131ms/step - loss: 0.4195 - val_loss: 0.3703
Epoch 4/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.4125 - val_loss: 0.4016
Epoch 5/5000
40/40 [==============================] - 5s 117ms/step - loss: 0.4357 - val_loss: 0.4544
Epoch 6/5000
40/40 [==============================] - 5s 117ms/step - loss: 0.4450 - val_loss: 0.3958
Epoch 7/5000
40/40 [==============================] - 5s 117ms/step - loss: 0.4433 - val_loss: 0.3999
Epoch 8/5000
40/40 [==============================] - 5s 117ms/step - loss: 0.4303 - val_loss: 0.4038
Epoch 9/5000
40/40 [==============================] - 5s 117ms/step - loss: 0.4096 - val_loss: 0.3589
Epoch 10/5000
40/40 [==============================] - 5s 117ms/step - loss: 0.428

KeyboardInterrupt: 

In [35]:
print(autoencoder_18.evaluate(X_test, X_test))

54/54 [==============================] - 1s 10ms/step - loss: 0.2165
0.21648064255714417


In [36]:
print(autoencoder_18_hier_2.ae.evaluate(X_test, X_test))

54/54 [==============================] - 1s 10ms/step - loss: 0.2165
0.21648064255714417


In [37]:
print(autoencoder_18_hier_2.evaluate(X_test, X_test))

54/54 [==============================] - 1s 12ms/step - loss: 0.1990
0.19895072281360626


In [38]:
training_history_hier_2 = autoencoder_18_hier_2.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=5000,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/5000
40/40 [==============================] - 5s 115ms/step - loss: 0.1733 - val_loss: 0.1954
Epoch 2/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.1688 - val_loss: 0.1914
Epoch 3/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.1753 - val_loss: 0.2012
Epoch 4/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.1741 - val_loss: 0.1925
Epoch 5/5000
40/40 [==============================] - 5s 120ms/step - loss: 0.1705 - val_loss: 0.1943
Epoch 6/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.1675 - val_loss: 0.1916
Epoch 7/5000
40/40 [==============================] - 5s 117ms/step - loss: 0.1713 - val_loss: 0.1929
Epoch 8/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.1724 - val_loss: 0.1921
Epoch 9/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.1725 - val_loss: 0.1996
Epoch 10/5000
40/40 [==============================] - 5s 116ms/step - loss: 0.172

In [39]:
autoencoder_18_hier_2.evaluate(X_test, X_test)

54/54 [==============================] - 1s 13ms/step - loss: 0.1843


0.18430177867412567

In [40]:
class Autoencoder_hier_3(Model):
    def __init__(self, encoder, decoder, ae):
        super(Autoencoder_hier_3, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder
        self.ae = ae
        self.ae.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        latent_1 = self.ae.ae.encoder(x)
        latent_2 = self.ae.encoder(x)
        latent_all = Concatenate(axis=-1)([encoded, latent_1, latent_2])
        decoded = self.decoder(latent_all)
        return decoded

In [41]:
encoder_18_sub3 = encoder(18)
print(encoder_18_sub3.summary())

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 192, 192, 2)]     0         
                                                                 
 conv2d_21 (Conv2D)          (None, 192, 192, 16)      304       
                                                                 
 batch_normalization_24 (Bat  (None, 192, 192, 16)     64        
 chNormalization)                                                
                                                                 
 leaky_re_lu_36 (LeakyReLU)  (None, 192, 192, 16)      0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 96, 96, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 96, 96, 16)        2320

In [42]:
decoder_18_sub3 = decoder(54)
print(decoder_18_sub3.summary())

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 54)]              0         
                                                                 
 dense_7 (Dense)             (None, 576)               31680     
                                                                 
 reshape_3 (Reshape)         (None, 3, 3, 64)          0         
                                                                 
 conv2d_transpose_18 (Conv2D  (None, 6, 6, 64)         36928     
 Transpose)                                                      
                                                                 
 batch_normalization_28 (Bat  (None, 6, 6, 64)         256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_42 (LeakyReLU)  (None, 6, 6, 64)          0   

In [44]:
autoencoder_18_hier_3 = Autoencoder_hier_3(encoder_18_sub3, decoder_18_sub3, autoencoder_18_hier_2)
autoencoder_18_hier_3.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [45]:
logdir = "autoencoder_18_hier_3/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_filepath = 'autoencoder_18_hier_3/ckp/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               save_freq=10*40)

In [46]:
training_history_hier_3 = autoencoder_18_hier_3.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=32,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/32
40/40 [==============================] - 7s 147ms/step - loss: 7.8148 - val_loss: 4.6423
Epoch 2/32
40/40 [==============================] - 5s 126ms/step - loss: 2.1391 - val_loss: 1.1884
Epoch 3/32
40/40 [==============================] - 5s 125ms/step - loss: 0.8561 - val_loss: 0.7132
Epoch 4/32
40/40 [==============================] - 5s 126ms/step - loss: 0.6642 - val_loss: 0.9077
Epoch 5/32
40/40 [==============================] - 5s 126ms/step - loss: 0.6625 - val_loss: 0.7915
Epoch 6/32
40/40 [==============================] - 5s 127ms/step - loss: 0.5819 - val_loss: 0.6588
Epoch 7/32
40/40 [==============================] - 5s 125ms/step - loss: 0.5604 - val_loss: 0.5007
Epoch 8/32
40/40 [==============================] - 5s 126ms/step - loss: 0.5223 - val_loss: 0.5794
Epoch 9/32
40/40 [==============================] - 5s 126ms/step - loss: 0.4866 - val_loss: 0.4863
Epoch 10/32
40/40 [==============================] - 5s 130ms/step - loss: 0.5133 - val_loss: 0.5413

In [47]:
autoencoder_18.evaluate(X_test, X_test)

54/54 [==============================] - 1s 11ms/step - loss: 0.2165


0.21648064255714417

In [48]:
autoencoder_18_hier_2.evaluate(X_test, X_test)

54/54 [==============================] - 1s 12ms/step - loss: 0.1843


0.18430177867412567

In [49]:
training_history_hier_3 = autoencoder_18_hier_3.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=5000,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/5000
40/40 [==============================] - 6s 140ms/step - loss: 0.3883 - val_loss: 0.2937
Epoch 2/5000
40/40 [==============================] - 5s 125ms/step - loss: 0.3690 - val_loss: 0.3161
Epoch 3/5000
40/40 [==============================] - 5s 126ms/step - loss: 0.3860 - val_loss: 0.2999
Epoch 4/5000
40/40 [==============================] - 5s 126ms/step - loss: 0.3533 - val_loss: 0.2833
Epoch 5/5000
40/40 [==============================] - 5s 126ms/step - loss: 0.3581 - val_loss: 0.2854
Epoch 6/5000
40/40 [==============================] - 5s 126ms/step - loss: 0.3661 - val_loss: 0.3746
Epoch 7/5000
40/40 [==============================] - 5s 126ms/step - loss: 0.3657 - val_loss: 0.3998
Epoch 8/5000
40/40 [==============================] - 5s 130ms/step - loss: 0.3775 - val_loss: 0.3336
Epoch 9/5000
40/40 [==============================] - 5s 126ms/step - loss: 0.3690 - val_loss: 0.2816
Epoch 10/5000
40/40 [==============================] - 5s 125ms/step - loss: 0.354

In [50]:
class Autoencoder_hier_4(Model):
    def __init__(self, encoder, decoder, ae):
        super(Autoencoder_hier_4, self).__init__() 
        self.encoder = encoder
        self.decoder = decoder
        self.ae = ae
        self.ae.trainable = False

    def call(self, x):
        encoded = self.encoder(x)
        latent_1 = self.ae.ae.ae.encoder(x)
        latent_2 = self.ae.ae.encoder(x)
        latent_3 = self.ae.encoder(x)
        latent_all = Concatenate(axis=-1)([encoded, latent_1, latent_2, latent_3])
        decoded = self.decoder(latent_all)
        return decoded

In [51]:
encoder_18_sub4 = encoder(18)
print(encoder_18_sub4.summary())

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 192, 192, 2)]     0         
                                                                 
 conv2d_28 (Conv2D)          (None, 192, 192, 16)      304       
                                                                 
 batch_normalization_32 (Bat  (None, 192, 192, 16)     64        
 chNormalization)                                                
                                                                 
 leaky_re_lu_48 (LeakyReLU)  (None, 192, 192, 16)      0         
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 96, 96, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 96, 96, 16)        2320

In [52]:
decoder_18_sub4 = decoder(72)
print(decoder_18_sub4.summary())

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 72)]              0         
                                                                 
 dense_9 (Dense)             (None, 576)               42048     
                                                                 
 reshape_4 (Reshape)         (None, 3, 3, 64)          0         
                                                                 
 conv2d_transpose_24 (Conv2D  (None, 6, 6, 64)         36928     
 Transpose)                                                      
                                                                 
 batch_normalization_36 (Bat  (None, 6, 6, 64)         256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_54 (LeakyReLU)  (None, 6, 6, 64)          0   

In [54]:
autoencoder_18_hier_4 = Autoencoder_hier_4(encoder_18_sub4, decoder_18_sub4, autoencoder_18_hier_3)
autoencoder_18_hier_4.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [55]:
logdir = "autoencoder_18_hier_4/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_filepath = 'autoencoder_18_hier_4/ckp/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               save_freq=10*40)

In [56]:
autoencoder_18.evaluate(X_test, X_test)

54/54 [==============================] - 1s 10ms/step - loss: 0.2165


0.21648064255714417

In [58]:
autoencoder_18_hier_2.evaluate(X_test, X_test)

54/54 [==============================] - 1s 13ms/step - loss: 0.1843


0.18430177867412567

In [59]:
autoencoder_18_hier_3.evaluate(X_test, X_test)

54/54 [==============================] - 1s 15ms/step - loss: 0.1687


0.1687164306640625

In [60]:
training_history_hier_4 = autoencoder_18_hier_4.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=32,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/32
40/40 [==============================] - 8s 160ms/step - loss: 7.3694 - val_loss: 5.4398
Epoch 2/32
40/40 [==============================] - 5s 138ms/step - loss: 4.1366 - val_loss: 2.8723
Epoch 3/32
40/40 [==============================] - 5s 138ms/step - loss: 1.2865 - val_loss: 0.9391
Epoch 4/32
40/40 [==============================] - 5s 137ms/step - loss: 0.7043 - val_loss: 0.6647
Epoch 5/32
40/40 [==============================] - 6s 138ms/step - loss: 0.6423 - val_loss: 0.6214
Epoch 6/32
40/40 [==============================] - 6s 138ms/step - loss: 0.6003 - val_loss: 0.5820
Epoch 7/32
40/40 [==============================] - 5s 138ms/step - loss: 0.5258 - val_loss: 0.4679
Epoch 8/32
40/40 [==============================] - 5s 138ms/step - loss: 0.4984 - val_loss: 0.5629
Epoch 9/32
40/40 [==============================] - 6s 138ms/step - loss: 0.4765 - val_loss: 0.4182
Epoch 10/32
40/40 [==============================] - 6s 142ms/step - loss: 0.4734 - val_loss: 0.3664

In [61]:
autoencoder_18.evaluate(X_test, X_test)

54/54 [==============================] - 1s 11ms/step - loss: 0.2165


0.21648064255714417

In [62]:
autoencoder_18_hier_2.evaluate(X_test, X_test)

54/54 [==============================] - 1s 12ms/step - loss: 0.1843


0.18430177867412567

In [63]:
autoencoder_18_hier_3.evaluate(X_test, X_test)

54/54 [==============================] - 1s 15ms/step - loss: 0.1687


0.1687164306640625

In [ ]:
training_history_hier_4 = autoencoder_18_hier_4.fit(X_train, X_train,
                                      batch_size=128,
                                      epochs=6000,
                                      shuffle=True,
                                      validation_data=(X_val, X_val),
                                      callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/6000
40/40 [==============================] - 6s 150ms/step - loss: 0.3680 - val_loss: 0.3097
Epoch 2/6000
40/40 [==============================] - 5s 137ms/step - loss: 0.3766 - val_loss: 0.3486
Epoch 3/6000
40/40 [==============================] - 5s 138ms/step - loss: 0.3747 - val_loss: 0.3191
Epoch 4/6000
40/40 [==============================] - 5s 138ms/step - loss: 0.3561 - val_loss: 0.2731
Epoch 5/6000
40/40 [==============================] - 5s 138ms/step - loss: 0.3452 - val_loss: 0.3157
Epoch 6/6000
40/40 [==============================] - 5s 138ms/step - loss: 0.3481 - val_loss: 0.2941
Epoch 7/6000
40/40 [==============================] - 5s 138ms/step - loss: 0.3500 - val_loss: 0.2642
Epoch 8/6000
40/40 [==============================] - 6s 143ms/step - loss: 0.3700 - val_loss: 0.3003
Epoch 9/6000
40/40 [==============================] - 6s 139ms/step - loss: 0.3468 - val_loss: 0.2827
Epoch 10/6000
40/40 [==============================] - 6s 138ms/step - loss: 0.347